In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import functions as f
from tqdm import tqdm

TTN_BIN = 4275
TTN_BINS = [4275, 4276, 4277, 4278]
chrom2_bins = [2490, 4911]

contact_matrix_zero = np.load('/Users/ethan/Desktop/chromatin-heart-dev/samples/contact_matrix_100kb_balanced_zeroed.npy')




# LOAD BIN MAP
bin_map = f.load_bin_map('/Users/ethan/Desktop/chromatin-heart-dev/data/bin_map_human_100000.bed')

gene_bins = []
with open('/Users/ethan/Desktop/chromatin-heart-dev/data/gene_bins.txt', 'r') as file:
    for line in file:
        gene_bins.append(line.strip())
gene_bins = [int(x) for x in gene_bins]


non_gene_bins = []
with open('/Users/ethan/Desktop/chromatin-heart-dev/data/non_gene_bins.txt', 'r') as file:
    for line in file:
        non_gene_bins.append(line.strip())
non_gene_bins = [int(x) for x in non_gene_bins]

In [ ]:

def build_walk_index(contact_matrix):
    """
    Precompute for each node:
      - neighbors[i]: 1D int array of neighbors
      - cdfs[i]:      1D float array of cumulative probabilities
    """
    N = contact_matrix.shape[0]
    neighbors = [None]*N
    cdfs      = [None]*N

    for i in tqdm(range(N)):
        w = contact_matrix[i]
        idx = np.nonzero(w)[0]
        if idx.size == 0:
            neighbors[i] = np.empty(0, dtype=int)
            cdfs[i]      = np.empty(0, dtype=float)
        else:
            probs = w[idx] / w[idx].sum()
            neighbors[i] = idx
            cdfs[i]      = np.cumsum(probs)
    return neighbors, cdfs

neighbors, cdfs = build_walk_index(contact_matrix_zero) 



In [ ]:
contact_matrix = np.load('/Users/ethan/Desktop/chromatin-heart-dev/samples/contact_matrix_100kb_balanced.npy')

## Finding TTN Clique Greedy

In [ ]:
TTN_BIN = 4275
ttn_clique = f.find_clique_greedy(contact_matrix, 5, TTN_BIN, bin_map)
G_TTN = f.clique_to_graph(contact_matrix, ttn_clique, TTN_BIN)
score_greedy = f.calculate_avg_interaction_strength(contact_matrix, ttn_clique)

In [ ]:
ttn_clique, score_greedy

## Sample Matrix

In [ ]:
import numpy as np
from scipy.linalg import eig
import math

def analytical_clique(contact_matrix: np.ndarray, n: int):
    # 1) Build P so that P[i,j] = prob of moving i -> j
    row_sums = contact_matrix.sum(axis=1)
    P = np.zeros_like(contact_matrix, dtype=float)
    for i, s in enumerate(row_sums):
        if s > 0:
            P[i, :] = contact_matrix[i, :] / s
        else:
            # no outgoing edges: stay in place
            P[i, i] = 1.0

    # 2) Compute left eigenvectors of P (i.e., eigenvectors of P^T)
    #    eig returns (w, vl) where vl[:,k] is the left eigenvector for w[k]
    w, vl = eig(P, left=True, right=False)

    # 3) Find the eigenvalue closest to 1
    idx = int(np.argmin(np.abs(w - 1.0)))
    pi = vl[:, idx].real  # take real part

    # 4) Normalize to sum to 1
    pi = pi / np.sum(pi)

    # 5) Pick top-n
    clique = np.argsort(pi)[-n:][::-1]
    return clique, pi

In [ ]:
import numpy as np

def analytical_diffusion_clique(contact_matrix: np.ndarray,
                                start_node: int,
                                n: int,
                                alpha: float = 0.1):

    N = contact_matrix.shape[0]

    # 1) Build the row‑stochastic transition matrix P
    P = np.zeros((N, N), dtype=float)
    row_sums = contact_matrix.sum(axis=1)
    for i in range(N):
        if row_sums[i] > 0:
            P[i, :] = contact_matrix[i, :] / row_sums[i]
        else:
            # no neighbors → self‑loop
            P[i, i] = 1.0

    # 2) Form the fundamental matrix: F = (I - (1-alpha)*P)^(-1)
    I = np.eye(N)
    F = np.linalg.inv(I - (1 - alpha) * P)

    # 3) Extract the expected visits for a start at `start_node`
    visits = F[start_node, :]

    # 4) Pick the top‑n nodes by descending visits
    clique = np.argsort(visits)[-n:][::-1]
    return clique, visits

In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from time import time   

# print time taken
def sparse_diffusion_clique(contact_matrix, start_node, n, alpha=0.1, rtol=1e-6):
    # 1) Ensure CSR sparse matrix
    if isinstance(contact_matrix, np.ndarray):
        contact_matrix = sp.csr_matrix(contact_matrix)
    else:
        contact_matrix = contact_matrix.tocsr()

    N = contact_matrix.shape[0]

    # 2) Normalize rows → P
    row_sums = np.array(contact_matrix.sum(axis=1)).ravel()
    P = contact_matrix.tolil()
    inv_row = np.zeros_like(row_sums, dtype=float)
    nonzero = row_sums > 0
    inv_row[nonzero] = 1.0 / row_sums[nonzero]

    for i in range(N):
        if nonzero[i]:
            P.data[i] = [val * inv_row[i] for val in P.data[i]]
        else:
            # self‑loop
            P.rows[i] = [i]
            P.data[i] = [1.0]

    P = P.tocsr()

    # 3) Build M = I - (1-α)P
    M = sp.eye(N, format="csr") - (1 - alpha) * P

    # 4) Setup RHS e_s
    e_s = np.zeros(N, float)
    e_s[start_node] = 1.0

    # 5) Solve (I - (1-α)P)^T v^T = e_s^T via GMRES
    visits, info = spla.gmres(M.T, e_s, rtol=rtol)
    if info != 0:
        raise RuntimeError(f"GMRES did not converge (info={info})")

    # 6) Top‑n
    clique = np.argsort(visits)[-n:][::-1]

    return clique

In [ ]:
# create a sample matrix
sample_matrix = f.generate_sample_matrix_bins(100)

# visualize the contact matrix hic 
plt.imshow(sample_matrix, cmap='hot', interpolation='nearest')
plt.show()


In [ ]:
analytical_diffusion_clique(sample_matrix, start_node=4, n=6)

In [ ]:
f.random_walk(sample_matrix, start_node=4, n=6, num_molecules=100000, alpha=0.1, verbose=True)  

In [ ]:
sparse_diffusion_clique(sample_matrix, start_node=4, n=6, alpha=0.1, rtol=1e-6)

In [ ]:
import time
import numpy as np
import pandas as pd


sizes = [100, 1000, 5000, 10000]
results = []

for N in sizes:
    # 1) generate
    mat = f.generate_sample_matrix_bins(N)
    
    row = {'N': N}
    
    # 2) analytical
    t0 = time.perf_counter()
    analytical_diffusion_clique(mat, start_node=4, n=6)
    row['analytical_s'] = time.perf_counter() - t0
    
    # 3) random walk
    t0 = time.perf_counter()
    f.random_walk(mat, start_node=4, n=6, num_molecules=10000, alpha=0.1, verbose=False)
    row['random_walk_s'] = time.perf_counter() - t0
    
    # 4) sparse
    t0 = time.perf_counter()
    sparse_diffusion_clique(mat, start_node=4, n=6, alpha=0.1, rtol=1e-6)
    row['sparse_s'] = time.perf_counter() - t0
    
    results.append(row)

# 5) summarize
df = pd.DataFrame(results).set_index('N')
print(df)

In [ ]:


# zero out cm if not above 2
sample_matrix = np.where(contact_matrix < 2, 0, sample_matrix)

# plot this on a graph
G = f.construct_graph_from_contact_matrix(sample_matrix, threshold=0)



## Random Walk Clique Size 5

In [ ]:
top_nodes = f.random_walk(contact_matrix_zero, TTN_BIN, 5, num_molecules=1000, alpha=0.05, verbose=True) 
rw_score_5  = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
rw_score_5  


In [ ]:
bg_model_random_walk_5 = f.create_background_model_rw(contact_matrix_zero, 5, num_molecules=1000, num_iterations=100, alpha=0.05)

In [ ]:
f.simple_p_test(rw_score_5, bg_model_random_walk_5)

In [ ]:
rw_score_5

## Random Walk Clique Size 5 with 100 molecules

In [ ]:
bg_model_random_walk_sample = f.create_background_model_rw(contact_matrix_zero, 5, num_molecules=100, num_iterations=1000, alpha=0.05)

In [ ]:
top_nodes = f.random_walk(contact_matrix_zero, TTN_BIN, 5, num_molecules=100, alpha=0.05, verbose=True) 
rw_score_5  = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
rw_score_5  


In [ ]:
f.simple_p_test(rw_score_5, bg_model_random_walk_sample)

In [ ]:
nodes = [ 4277, 30364, 16661,  6802,  4876]
f.clique_to_graph(contact_matrix_zero, nodes, TTN_BIN)   


## Random Walk Clique Size 10

In [ ]:
top_nodes = f.random_walk(contact_matrix_zero, TTN_BIN, 10, num_molecules=1000, alpha=0.05, verbose=True) 
rw_score_10  = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)

In [ ]:
bg_model_random_walk_10 = f.create_background_model_rw(contact_matrix_zero, 10, num_molecules=1000, num_iterations=100, alpha=0.05)
# save list of scores in /background_models

with open('/Users/ethan/Desktop/chromatin-heart-dev/background_models/rw_scores_10.txt', 'w') as f:
    for item in bg_model_random_walk_10:
        f.write("%s\n" % item)

In [ ]:
f.simple_p_test(rw_score_10, bg_model_random_walk_10)

In [ ]:
rw_score_10

## RW Clique Size 15

In [ ]:
bg_model_random_walk_15 = f.create_background_model_rw(contact_matrix_zero, 15, num_molecules=1000, num_iterations=1000, alpha=0.05)

In [ ]:
top_nodes_rw_15 = f.random_walk(contact_matrix_zero, TTN_BIN, 15, num_molecules=1000, alpha=0.05, verbose=True) 
rw_score_15  = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
print(top_nodes_rw_15)
print(rw_score_15)

In [ ]:
import functions as f
f.simple_p_test(rw_score_15, bg_model_random_walk_15)

## Plot of Clique Size to P Value

In [ ]:
from scipy import stats

def t_test(observed_score, score_list):
    t_stat, p_value = stats.ttest_1samp(score_list, observed_score)
    return p_value


def mann_whitney_u_test(ttn_score, random_scores):
    ttn_array = np.array([ttn_score] * len(random_scores)) 
    u_statistic, p_value = stats.mannwhitneyu(ttn_array, random_scores, alternative='greater')
    return p_value

def empirical_p_value(observed_score, random_scores):
    count = sum(score >= observed_score for score in random_scores)
    p_value = count / len(random_scores)
    return p_value

    """
    Compares an observed score to a list of scores using a permutation test.
    """
    combined_scores = np.concatenate([score_list, [observed_score]])
    n_observed = 1
    n_other = len(score_list)

    observed_statistic = observed_score # Or another relevant statistic

    extreme_count = 0
    for _ in range(n_permutations):
        permuted_indices = np.random.permutation(len(combined_scores))
        permuted_observed_score = combined_scores[permuted_indices[-1]] # Assume observed score is the last one

        permuted_statistic = permuted_observed_score # Use the same statistic

        # Adjust the comparison based on the direction of significance you're looking for
        # For "more significant" (assuming higher score), check if permuted statistic is as extreme or more extreme
        if permuted_statistic >= observed_statistic:
            extreme_count += 1

    p_value = extreme_count / n_permutations
    return p_value


In [ ]:
TTN_BIN

In [ ]:
from tqdm import tqdm

clique_sizes = [3, 4, 5, 8, 10, 12, 15, 20, 40, 60]
p_values = []

for i in clique_sizes:
    top_nodes = f.find_clique_greedy(contact_matrix_zero, i, TTN_BIN, bin_map)
    greedy_score = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
    bg_model_greedy = f.create_background_model_greedy_strong(contact_matrix_zero, i, bin_map, gene_bins=gene_bins, num_iterations=1000)
    
    p_value = mann_whitney_u_test(greedy_score, bg_model_greedy) 
    p_values.append(p_value)

plt.plot(clique_sizes, p_values, marker='o')
plt.xlabel('Clique Size')
plt.ylabel('p-value')
plt.yscale('log')  # Useful if values vary widely
plt.axhline(0.05, color='r', linestyle='--', label='Significance Threshold (0.05)')
plt.legend()
plt.show()

## Distribution Plot of Backgrounds

In [ ]:
TTN_BIN

In [ ]:
test = f.find_clique_greedy(contact_matrix_zero, 5, 4276, bin_map)
f.clique_to_graph(contact_matrix_zero, test, TTN_BIN)

In [ ]:
f.calculate_avg_interaction_strength(contact_matrix_zero, test)

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

clique_sizes = [5, 10, 20, 40]
p_values = []
num_iterations = 1000
ttn_bins = [4275, 4276, 4277, 4278]
colors = ['red', 'blue', 'green', 'orange']  # Define different colors for each bin

for i in clique_sizes:
    ttn_cliques = [f.find_clique_greedy(contact_matrix_zero, i, ttn_bin, bin_map) for ttn_bin in ttn_bins]
    ttn_scores = [f.calculate_avg_interaction_strength(contact_matrix_zero, ttn_clique) for ttn_clique in ttn_cliques]
    
    bg_model_greedy_weak = f.create_background_model_greedy(contact_matrix_zero, i, bin_map, non_gene_bins, label='weak', num_iterations=num_iterations, display=False)
    bg_model_greedy_strong = f.create_background_model_greedy(contact_matrix_zero, i, bin_map, gene_bins, label='strong', num_iterations=num_iterations, display=False)

    # Create subplots: 1 row, 2 columns
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Plot weak model
    axes[0].hist(bg_model_greedy_weak, bins=50, color='skyblue', edgecolor='black')
    for idx, (ttn_score, ttn_bin) in enumerate(zip(ttn_scores, ttn_bins)):
        axes[0].axvline(x=ttn_score, color=colors[idx], linestyle='--', label=f'TTN Score {ttn_bin}: {ttn_score}')
    axes[0].set_xlabel('Average Interaction Score')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(f'Distribution of AIS using Greedy Weak for {num_iterations} Weak Cliques of Size {i}')
    axes[0].legend()

    # Plot strong model
    axes[1].hist(bg_model_greedy_strong, bins=50, color='skyblue', edgecolor='black')
    for idx, (ttn_score, ttn_bin) in enumerate(zip(ttn_scores, ttn_bins)):
        axes[1].axvline(x=ttn_score, color=colors[idx], linestyle='--', label=f'TTN Score {ttn_bin}: {ttn_score}')
    axes[1].set_xlabel('Average Interaction Score')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'Distribution of AIS using Greedy for {num_iterations} Strong Cliques of Size {i}')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

clique_sizes = [20]
p_values = []
num_iterations = 1000
ttn_bins = [4275, 4276, 4277, 4278]
colors = ['red', 'blue', 'green', 'orange']  # Define different colors for each bin

for i in clique_sizes:
    ttn_cliques = [f.find_clique_greedy(contact_matrix_zero, i, ttn_bin, bin_map) for ttn_bin in ttn_bins]
    ttn_scores = [f.calculate_avg_interaction_strength(contact_matrix_zero, ttn_clique) for ttn_clique in ttn_cliques]
    
    bg_model_greedy_weak = f.create_background_model_greedy(contact_matrix_zero, i, bin_map, non_gene_bins, label='weak', num_iterations=num_iterations, display=False)
    bg_model_greedy_strong = f.create_background_model_greedy(contact_matrix_zero, i, bin_map, gene_bins, label='strong', num_iterations=num_iterations, display=False)
    
    # Create subplots: 1 row, 2 columns
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Plot weak model
    axes[0].hist(bg_model_greedy_weak, bins=50, color='skyblue', edgecolor='black')
    for idx, (ttn_score, ttn_bin) in enumerate(zip(ttn_scores, ttn_bins)):
        axes[0].axvline(x=ttn_score, color=colors[idx], linestyle='--', label=f'TTN Score {ttn_bin}: {ttn_score}')
    axes[0].set_xlabel('Average Interaction Score')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(f'Distribution of AIS using Greedy Weak for {num_iterations} Weak Cliques of Size {i}')
    axes[0].legend()

    # Plot strong model
    axes[1].hist(bg_model_greedy_strong, bins=50, color='skyblue', edgecolor='black')
    for idx, (ttn_score, ttn_bin) in enumerate(zip(ttn_scores, ttn_bins)):
        axes[1].axvline(x=ttn_score, color=colors[idx], linestyle='--', label=f'TTN Score {ttn_bin}: {ttn_score}')
    axes[1].set_xlabel('Average Interaction Score')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'Distribution of AIS using Greedy for {num_iterations} Strong Cliques of Size {i}')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

clique_sizes = [2]
p_values = []
num_iterations = 500
ttn_bins = [4275, 4276, 4277, 4278]
colors = ['red', 'blue', 'green', 'orange']  # Define different colors for each bin

for i in clique_sizes:
    ttn_cliques = [f.find_clique_greedy(contact_matrix_zero, i, ttn_bin, bin_map) for ttn_bin in ttn_bins]
    ttn_scores = [f.calculate_avg_interaction_strength(contact_matrix_zero, ttn_clique) for ttn_clique in ttn_cliques]
    
    bg_model_greedy_weak = f.create_background_model_greedy(contact_matrix_zero, i, bin_map, non_gene_bins, label='weak', num_iterations=num_iterations, display=False)
    bg_model_greedy_strong = f.create_background_model_greedy(contact_matrix_zero, i, bin_map, gene_bins, label='strong', num_iterations=num_iterations, display=False)

    # Create subplots: 1 row, 2 columns
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Plot weak model
    axes[0].hist(bg_model_greedy_weak, bins=50, color='skyblue', edgecolor='black')
    for idx, (ttn_score, ttn_bin) in enumerate(zip(ttn_scores, ttn_bins)):
        axes[0].axvline(x=ttn_score, color=colors[idx], linestyle='--', label=f'TTN Score {ttn_bin}: {ttn_score}')
    axes[0].set_xlabel('Average Interaction Score')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(f'Distribution of AIS using Greedy Weak for {num_iterations} Weak Cliques of Size {i}')
    axes[0].legend()

    # Plot strong model
    axes[1].hist(bg_model_greedy_strong, bins=50, color='skyblue', edgecolor='black')
    for idx, (ttn_score, ttn_bin) in enumerate(zip(ttn_scores, ttn_bins)):
        axes[1].axvline(x=ttn_score, color=colors[idx], linestyle='--', label=f'TTN Score {ttn_bin}: {ttn_score}')
    axes[1].set_xlabel('Average Interaction Score')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'Distribution of AIS using Greedy for {num_iterations} Strong Cliques of Size {i}')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

## Violin Plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

clique_sizes = [5, 10, 20]

bg_scores_list = []
for size in clique_sizes:
    path = f'/Users/ethan/Desktop/chromatin-heart-dev/background_models/greedy_scores_{size}_iterations_1000_strong.txt'
    with open(path, 'r') as file:
        scores = [float(line) for line in file]
    bg_scores_list.append(scores)

# Create the violin plot
plt.figure(figsize=(8, 5))
sns.violinplot(data=bg_scores_list, inner="quartile", cut=0)
plt.xticks(ticks=range(len(clique_sizes)), labels=clique_sizes)
plt.xlabel('Clique Size')
plt.ylabel('Background Score Distribution')
plt.title('Violin Plot of Greedy‐Score Strong Background Distributions')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

clique_sizes = [5, 10, 20]

bg_scores_list = []
for size in clique_sizes:
    path = f'/Users/ethan/Desktop/chromatin-heart-dev/background_models/greedy_scores_{size}_iterations_1000_weak.txt'
    with open(path, 'r') as f:
        scores = [float(line) for line in f]
    bg_scores_list.append(scores)

# Create the violin plot
plt.figure(figsize=(8, 5))
sns.violinplot(data=bg_scores_list, inner="quartile", cut=0)
plt.xticks(ticks=range(len(clique_sizes)), labels=clique_sizes)
plt.xlabel('Clique Size')
plt.ylabel('Background Score Distribution')
plt.title('Violin Plot of Greedy‐Score Weak Background Distributions')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

clique_sizes = [5, 10, 20]
records = []

for model_type in ["strong", "weak"]:
    for size in clique_sizes:
        path = (f"/Users/ethan/Desktop/chromatin-heart-dev/"
                f"background_models/greedy_scores_{size}_iterations_1000_{model_type}.txt")
        with open(path, "r") as f:
            scores = [float(line) for line in f]
        for s in scores:
            records.append({
                "Clique Size": size,
                "Score": s,
                "Model": "Strong" if model_type == "strong" else "Weak"
            })

df = pd.DataFrame.from_records(records)

# 2) Plot a split‐violin
plt.figure(figsize=(8, 5))
sns.violinplot(
    data=df,
    x="Clique Size",
    y="Score",
    hue="Model",
    split=True,            # split each violin in half
    inner="quartile",      # show median + quartiles
    cut=0                  # don’t extend past data
)
sns.despine(left=True)
plt.title("Strong vs. Weak Model Background Distributions")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


clique_sizes = [5, 10, 20]
records = []
for model_type in ["strong", "weak"]:
    for size in clique_sizes:
        path = (
            f"/Users/ethan/Desktop/chromatin-heart-dev/"
            f"background_models/greedy_scores_{size}_iterations_1000_{model_type}.txt"
        )
        with open(path, "r") as f:
            scores = [float(line) for line in f]
        for s in scores:
            records.append({
                "Clique Size": size,
                "Score": s,
                "Model": "Strong" if model_type=="strong" else "Weak"
            })



df = pd.DataFrame.from_records(records)

# ——— Plot with extra width ———
plt.figure(figsize=(14, 5))    # make it wider!
ax = sns.violinplot(
    data=df,
    x="Clique Size",
    y="Score",
    hue="Model",
    dodge=True,
    width=0.7,                # controls how “fat” each violin is
    inner="quartile",
    cut=0
)
sns.despine(left=True)
ax.set_yscale('symlog', linthresh=0.01)
plt.title("Strong vs. Weak Distributions (wider figure)")
plt.tight_layout()
plt.show()

## 5 Violin Plot

In [ ]:
# LOAD BIN MAP
bin_map_loc = f.load_bin_map_loc('/Users/ethan/Desktop/chromatin-heart-dev/data/bin_map_human_100000.bed')

In [ ]:
import os

def get_top_n_tf_clique(clique_size):
    # Load the transcription factor binding sites
    data_dir = './data/transcription_factors'
    tf_files = os.listdir(data_dir)
    interaction_scores = []

    # Contains top 1 bin for each TF
    max_bins = []
    bin_set = set()

    # Return counts of transcription factor binding sites in each 100kb bin
    def process_bed_file(file_path):
        bin_counts = {}

        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('#') or not line.strip():
                    continue
                columns = line.split()
                
                chrom = columns[0]
                start_position = int(columns[1])    
                
                bin_number = f.find_bin(chrom, start_position, bin_map_loc)
                
                if bin_number not in bin_counts:
                    bin_counts[bin_number] = 0
                bin_counts[bin_number] += 1

        return bin_counts



    for filename in os.listdir(data_dir):
        # open file

        try:    
            bin_counts = process_bed_file(os.path.join(data_dir, filename))
        except:
            print(f'Error processing {filename}')
            continue

        # get top 20 bins
        top_bins = sorted(bin_counts, key=bin_counts.get, reverse=True)[:clique_size]

        # print the top bins and corresponding counts
        print(f'Transcription factor: {filename}')
        print(f'Top {clique_size} bins: {top_bins}')
        print(f'Counts: {[bin_counts[bin] for bin in top_bins]}')

        max_bins.append(top_bins[0])
        bin_set.update(top_bins)

        interaction_strength = f.calculate_avg_interaction_strength(contact_matrix_zero, top_bins)
        interaction_scores.append(interaction_strength)

    return interaction_scores
    

In [ ]:
import os

def get_middle_n_tf_clique(clique_size):
    # Load the transcription factor binding sites
    data_dir = './data/transcription_factors'
    tf_files = os.listdir(data_dir)
    interaction_scores = []

    # Contains middle bins for each TF
    middle_bins = []
    bin_set = set()

    # Return counts of transcription factor binding sites in each 100kb bin
    def process_bed_file(file_path):
        bin_counts = {}

        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('#') or not line.strip():
                    continue
                columns = line.split()
                
                chrom = columns[0]
                start_position = int(columns[1])    
                
                bin_number = f.find_bin(chrom, start_position, bin_map_loc)
                
                if bin_number not in bin_counts:
                    bin_counts[bin_number] = 0
                bin_counts[bin_number] += 1

        return bin_counts

    for filename in os.listdir(data_dir):
        try:    
            bin_counts = process_bed_file(os.path.join(data_dir, filename))
        except:
            print(f'Error processing {filename}')
            continue

 
        sorted_bins = sorted(bin_counts.items(), key=lambda x: x[1], reverse=True)

        total_bins = len(sorted_bins)
        
        start_index = (total_bins - clique_size) // 2
        if start_index < 0:  
            start_index = 0
        
        middle_n_bins = [bin_num for bin_num, _ in sorted_bins[start_index:start_index + clique_size]]

        # print middle n bins and corresponding counts
        print(f'Transcription factor: {filename}')
        print(f'Middle {clique_size} bins: {middle_n_bins}')
        print(f'Counts: {[bin_counts[bin] for bin in middle_n_bins]}')
        
        
        if len(middle_n_bins) < clique_size:
            middle_n_bins = [bin_num for bin_num, _ in sorted_bins]
        
        if middle_n_bins: 
            middle_bins.append(middle_n_bins[0])
            bin_set.update(middle_n_bins)

            interaction_strength = f.calculate_avg_interaction_strength(contact_matrix_zero, middle_n_bins)
            interaction_scores.append(interaction_strength)

    return interaction_scores

In [ ]:
import functions as f

# do greedy on the top bins of each transcription factor
def bg_greedy_top_bin_tf(contact_matrix_zero, max_bins, clique_size, bin_map):
    interaction_scores = []
    for bin in tqdm(max_bins):
        top_nodes = f.find_clique_greedy(contact_matrix_zero, clique_size, bin, bin_map)
        greedy_score  = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
        interaction_scores.append(greedy_score)

    return interaction_scores

In [ ]:
test1 = get_middle_n_tf_clique(2)

In [ ]:
test2 = get_top_n_tf_clique(2)

In [ ]:
np.mean(test1), np.mean(test2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import functions as f

def get_top_n_tf_clique(clique_size):
    # Load the transcription factor binding sites
    data_dir = './data/transcription_factors'
    tf_files = os.listdir(data_dir)
    interaction_scores = []

    # Contains top 1 bin for each TF
    max_bins = []
    bin_set = set()

    # Return counts of transcription factor binding sites in each 100kb bin
    def process_bed_file(file_path):
        bin_counts = {}

        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('#') or not line.strip():
                    continue
                columns = line.split()
                
                chrom = columns[0]
                start_position = int(columns[1])    
                
                bin_number = f.find_bin(chrom, start_position, bin_map_loc)
                
                if bin_number not in bin_counts:
                    bin_counts[bin_number] = 0
                bin_counts[bin_number] += 1

        return bin_counts



    for filename in os.listdir(data_dir):
        # open file

        try:    
            bin_counts = process_bed_file(os.path.join(data_dir, filename))
        except:
            print(f'Error processing {filename}')
            continue

        # get top 20 bins
        top_bins = sorted(bin_counts, key=bin_counts.get, reverse=True)[:clique_size]

        max_bins.append(top_bins[0])
        bin_set.update(top_bins)

        interaction_strength = f.calculate_avg_interaction_strength(contact_matrix_zero, top_bins)
        interaction_scores.append(interaction_strength)

    return interaction_scores
    
# do greedy on the top bins of each transcription factor
def bg_greedy_top_bin_tf(contact_matrix_zero, max_bins, clique_size, bin_map):
    interaction_scores = []
    for bin in tqdm(max_bins):
        top_nodes = f.find_clique_greedy(contact_matrix_zero, clique_size, bin, bin_map)
        greedy_score  = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
        interaction_scores.append(greedy_score)

    return interaction_scores



# ——— parameters & record collection ———
clique_sizes = [5, 10, 20]
records = []

# 1) load your precomputed “strong”/“weak” scores from disk
for model_type in ["strong", "weak"]:
    label = "Strong" if model_type == "strong" else "Weak"
    for size in clique_sizes:
        path = (
            f"/Users/ethan/Desktop/chromatin-heart-dev/"
            f"background_models/greedy_scores_{size}_iterations_1000_{model_type}.txt"
        )
        with open(path, "r") as fh:
            scores = [float(line) for line in fh]
        for s in scores:
            records.append({
                "Clique Size": size,
                "Score": s,
                "Model": label
            })

# 2) now call your TF‐function for each clique size and append those too
for size in clique_sizes:
    tf_scores = get_top_n_tf_clique(size)
    for s in tf_scores:
        records.append({
            "Clique Size": size,
            "Score": s,
            "Model": "TF"
        })

# 3) call bg_greedy_top_bin_tf for each clique size and append those too
for size in clique_sizes:
    bg_scores = bg_greedy_top_bin_tf(contact_matrix_zero, max_bins, size, bin_map)
    for s in bg_scores:
        records.append({
            "Clique Size": size,
            "Score": s,
            "Model": "Greedy TF Top"
        })

# build the DataFrame
df = pd.DataFrame.from_records(records)

# ——— plot ———
plt.figure(figsize=(14, 5))
ax = sns.violinplot(
    data=df,
    x="Clique Size",
    y="Score",
    hue="Model",      # will now show Strong, Weak, TF
    dodge=True,
    width=0.7,
    inner="quartile",
    cut=0
)
sns.despine(left=True)
ax.set_yscale('symlog', linthresh=0.01)
ax.set_title("Strong vs. Weak vs. TF Distributions")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import functions as f

def get_top_n_tf_clique(clique_size):
    # Load the transcription factor binding sites
    data_dir = './data/transcription_factors'
    tf_files = os.listdir(data_dir)
    interaction_scores = []

    # Contains top 1 bin for each TF
    max_bins = []
    bin_set = set()

    # Return counts of transcription factor binding sites in each 100kb bin
    def process_bed_file(file_path):
        bin_counts = {}

        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('#') or not line.strip():
                    continue
                columns = line.split()
                
                chrom = columns[0]
                start_position = int(columns[1])    
                
                bin_number = f.find_bin(chrom, start_position, bin_map_loc)
                
                if bin_number not in bin_counts:
                    bin_counts[bin_number] = 0
                bin_counts[bin_number] += 1

        return bin_counts



    for filename in os.listdir(data_dir):
        # open file

        try:    
            bin_counts = process_bed_file(os.path.join(data_dir, filename))
        except:
            print(f'Error processing {filename}')
            continue

        # get top 20 bins
        top_bins = sorted(bin_counts, key=bin_counts.get, reverse=True)[:clique_size]

        max_bins.append(top_bins[0])
        bin_set.update(top_bins)

        interaction_strength = f.calculate_avg_interaction_strength(contact_matrix_zero, top_bins)
        interaction_scores.append(interaction_strength)

    return interaction_scores

def get_middle_n_tf_clique(clique_size):
    # Load the transcription factor binding sites
    data_dir = './data/transcription_factors'
    tf_files = os.listdir(data_dir)
    interaction_scores = []

    # Contains middle bins for each TF
    middle_bins = []
    bin_set = set()

    # Return counts of transcription factor binding sites in each 100kb bin
    def process_bed_file(file_path):
        bin_counts = {}

        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('#') or not line.strip():
                    continue
                columns = line.split()
                
                chrom = columns[0]
                start_position = int(columns[1])    
                
                bin_number = f.find_bin(chrom, start_position, bin_map_loc)
                
                if bin_number not in bin_counts:
                    bin_counts[bin_number] = 0
                bin_counts[bin_number] += 1

        return bin_counts

    for filename in os.listdir(data_dir):
        try:    
            bin_counts = process_bed_file(os.path.join(data_dir, filename))
        except:
            print(f'Error processing {filename}')
            continue

 
        sorted_bins = sorted(bin_counts.items(), key=lambda x: x[1], reverse=True)

        total_bins = len(sorted_bins)
        
        start_index = (total_bins - clique_size) // 2
        if start_index < 0:  
            start_index = 0
        
        middle_n_bins = [bin_num for bin_num, _ in sorted_bins[start_index:start_index + clique_size]]

        
        if len(middle_n_bins) < clique_size:
            middle_n_bins = [bin_num for bin_num, _ in sorted_bins]
        
        if middle_n_bins: 
            middle_bins.append(middle_n_bins[0])
            bin_set.update(middle_n_bins)

            interaction_strength = f.calculate_avg_interaction_strength(contact_matrix_zero, middle_n_bins)
            interaction_scores.append(interaction_strength)

    return interaction_scores

def bg_greedy_top_bin_tf(contact_matrix_zero, max_bins, clique_size, bin_map):
    interaction_scores = []
    for bin in tqdm(max_bins):
        top_nodes = f.find_clique_greedy(contact_matrix_zero, clique_size, bin, bin_map)
        greedy_score  = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
        interaction_scores.append(greedy_score)

    return interaction_scores


# ——— parameters & record collection ———
clique_sizes = [5, 10, 20]
records = []

# 1) load precomputed “strong”/“weak” bg scores from disk
# for model_type in ["strong", "weak"]:
#     label = "Strong" if model_type == "strong" else "Weak"
#     for size in clique_sizes:
#         path = (
#             f"/Users/ethan/Desktop/chromatin-heart-dev/"
#             f"background_models/greedy_scores_{size}_iterations_1000_{model_type}.txt"
#         )
#         with open(path, "r") as fh:
#             scores = [float(line) for line in fh]
#         for s in scores:
#             records.append({
#                 "Clique Size": size,
#                 "Score": s,
#                 "Model": label
#             })

# 2) now call get_top_n_tf_clique for each clique size and append those too
for size in clique_sizes:
    tf_scores = get_top_n_tf_clique(size)
    for s in tf_scores:
        records.append({
            "Clique Size": size,
            "Score": s,
            "Model": "Top TF"
        })

# 3) call get_middle_n_tf_clique for each clique size and append those too
for size in clique_sizes:
    middle_scores = get_middle_n_tf_clique(size)
    for s in middle_scores:
        records.append({
            "Clique Size": size,
            "Score": s,
            "Model": "Middle TF"
        })

# 4) call bg_greedy_top_bin_tf for each clique size and append those too
# for size in clique_sizes:
#     bg_scores = bg_greedy_top_bin_tf(contact_matrix_zero, max_bins, size, bin_map)
#     for s in bg_scores:
#         records.append({
#             "Clique Size": size,
#             "Score": s,
#             "Model": "Greedy TF Top"
#         })

# build the DataFrame
df = pd.DataFrame.from_records(records)

# ——— plot ———
plt.figure(figsize=(14, 5))
ax = sns.violinplot(
    data=df,
    x="Clique Size",
    y="Score",
    hue="Model",      # will now show Strong, Weak, TF
    dodge=True,
    width=0.7,
    inner="quartile",
    cut=0
)
sns.despine(left=True)
ax.set_yscale('symlog', linthresh=0.01)
ax.set_title("Top vs Middle TF Distributions")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import functions as f

def get_top_n_tf_clique(clique_size):
    # Load the transcription factor binding sites
    data_dir = './data/transcription_factors'
    tf_files = os.listdir(data_dir)
    interaction_scores = []

    # Contains top 1 bin for each TF
    max_bins = []
    bin_set = set()

    # Return counts of transcription factor binding sites in each 100kb bin
    def process_bed_file(file_path):
        bin_counts = {}

        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('#') or not line.strip():
                    continue
                columns = line.split()
                
                chrom = columns[0]
                start_position = int(columns[1])    
                
                bin_number = f.find_bin(chrom, start_position, bin_map_loc)
                
                if bin_number not in bin_counts:
                    bin_counts[bin_number] = 0
                bin_counts[bin_number] += 1

        return bin_counts



    for filename in os.listdir(data_dir):
        # open file

        try:    
            bin_counts = process_bed_file(os.path.join(data_dir, filename))
        except:
            print(f'Error processing {filename}')
            continue

        # get top 20 bins
        top_bins = sorted(bin_counts, key=bin_counts.get, reverse=True)[:clique_size]

        max_bins.append(top_bins[0])
        bin_set.update(top_bins)

        interaction_strength = f.calculate_avg_interaction_strength(contact_matrix_zero, top_bins)
        interaction_scores.append(interaction_strength)

    return interaction_scores

def get_middle_n_tf_clique(clique_size):
    # Load the transcription factor binding sites
    data_dir = './data/transcription_factors'
    tf_files = os.listdir(data_dir)
    interaction_scores = []

    # Contains middle bins for each TF
    middle_bins = []
    bin_set = set()

    # Return counts of transcription factor binding sites in each 100kb bin
    def process_bed_file(file_path):
        bin_counts = {}

        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('#') or not line.strip():
                    continue
                columns = line.split()
                
                chrom = columns[0]
                start_position = int(columns[1])    
                
                bin_number = f.find_bin(chrom, start_position, bin_map_loc)
                
                if bin_number not in bin_counts:
                    bin_counts[bin_number] = 0
                bin_counts[bin_number] += 1

        return bin_counts

    for filename in os.listdir(data_dir):
        try:    
            bin_counts = process_bed_file(os.path.join(data_dir, filename))
        except:
            print(f'Error processing {filename}')
            continue

 
        sorted_bins = sorted(bin_counts.items(), key=lambda x: x[1], reverse=True)

        total_bins = len(sorted_bins)
        
        start_index = (total_bins - clique_size) // 2
        if start_index < 0:  
            start_index = 0
        
        middle_n_bins = [bin_num for bin_num, _ in sorted_bins[start_index:start_index + clique_size]]

        
        if len(middle_n_bins) < clique_size:
            middle_n_bins = [bin_num for bin_num, _ in sorted_bins]
        
        if middle_n_bins: 
            middle_bins.append(middle_n_bins[0])
            bin_set.update(middle_n_bins)

            interaction_strength = f.calculate_avg_interaction_strength(contact_matrix_zero, middle_n_bins)
            interaction_scores.append(interaction_strength)

    return interaction_scores

def bg_greedy_top_bin_tf(contact_matrix_zero, max_bins, clique_size, bin_map):
    interaction_scores = []
    for bin in tqdm(max_bins):
        top_nodes = f.find_clique_greedy_fast(contact_matrix_zero, clique_size, bin)
        greedy_score  = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
        interaction_scores.append(greedy_score)

    return interaction_scores


# ——— parameters & record collection ———
clique_sizes = [5, 10, 20]
records = []

# 1) load precomputed “strong”/“weak” bg scores from disk
for model_type in ["strong", "weak"]:
    label = "Strong" if model_type == "strong" else "Weak"
    for size in clique_sizes:
        path = (
            f"/Users/ethan/Desktop/chromatin-heart-dev/"
            f"background_models/greedy_scores_{size}_iterations_1000_{model_type}.txt"
        )
        with open(path, "r") as fh:
            scores = [float(line) for line in fh]
        for s in scores:
            records.append({
                "Clique Size": size,
                "Score": s,
                "Model": label
            })

# 2) now call get_top_n_tf_clique for each clique size and append those too
for size in clique_sizes:
    tf_scores = get_top_n_tf_clique(size)
    for s in tf_scores:
        records.append({
            "Clique Size": size,
            "Score": s,
            "Model": "Top TF"
        })

# 3) call get_middle_n_tf_clique for each clique size and append those too
for size in clique_sizes:
    middle_scores = get_middle_n_tf_clique(size)
    for s in middle_scores:
        records.append({
            "Clique Size": size,
            "Score": s,
            "Model": "Middle TF"
        })

# 4) call bg_greedy_top_bin_tf for each clique size and append those too
for size in clique_sizes:
    bg_scores = bg_greedy_top_bin_tf(contact_matrix_zero, max_bins, size, bin_map)
    for s in bg_scores:
        records.append({
            "Clique Size": size,
            "Score": s,
            "Model": "Greedy TF Top"
        })

# build the DataFrame
df = pd.DataFrame.from_records(records)

# ——— plot ———
plt.figure(figsize=(14, 5))
ax = sns.violinplot(
    data=df,
    x="Clique Size",
    y="Score",
    hue="Model",      # will now show Strong, Weak, TF
    dodge=True,
    width=0.7,
    inner="quartile",
    cut=0
)
sns.despine(left=True)
ax.set_yscale('symlog', linthresh=0.01)
ax.set_title("Top vs Middle TF Distributions")
plt.tight_layout()
plt.show()

## More Violin Plot Stuff

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PolyCollection


clique_sizes = [5, 10, 20]
models       = ["strong", "weak"]
records      = []
greedy_scores = []

# 2) Compute the real interaction strength once per size,
#    and load each background distribution
for size in clique_sizes:
    # compute on your real contact matrix
    top_nodes = f.find_clique_greedy(contact_matrix_zero, size, TTN_BIN, bin_map)
    gs = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
    greedy_scores.append(gs)

    # load the null distributions
    for m in models:
        path = (
            f"/Users/ethan/Desktop/chromatin-heart-dev/"
            f"background_models/greedy_scores_{size}_iterations_1000_{m}.txt"
        )
        with open(path, "r") as bg:
            bg_scores = [float(line) for line in bg]
        for s in bg_scores:
            records.append({
                "Clique Size": size,
                "Score":       s,
                "Model":       m.capitalize()
            })

# 3) Build a DataFrame
df = pd.DataFrame.from_records(records)

# 4) Draw the side‐by‐side violins
fig, ax = plt.subplots(figsize=(10, 6))
sns.violinplot(
    data=df,
    x="Clique Size",
    y="Score",
    hue="Model",
    dodge=True,
    inner="quartile",
    cut=0,
    ax=ax
)

# 5) Overlay a horizontal line at each greedy_score
bodies   = [c for c in ax.collections if isinstance(c, PolyCollection)]
n_models = len(models)  # 2 in this case

for j, size in enumerate(clique_sizes):
    # the two violins for this size occupy positions j*2 and j*2+1 in `bodies`
    group = bodies[j*n_models:(j+1)*n_models]

    # collect all x‐coords across both violins
    xs = []
    for body in group:
        verts = body.get_paths()[0].vertices
        xs.extend(verts[:,0])
    x_min, x_max = min(xs), max(xs)

    y = greedy_scores[j]
    # draw the line
    ax.hlines(y, x_min, x_max, colors="red", linestyles="--")
    # annotate the exact value
    ax.text(
        (x_min + x_max)/2, y,
        y,
        ha="center", va="bottom",
        color="red", fontweight="bold"
    )

sns.despine(left=True)
plt.title("Strong vs. Weak Background Distributions\nwith Real Interaction Strength")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PolyCollection

clique_sizes = [5, 10, 20]
models = ["strong", "weak"]

greedy_scores = [
    f.calculate_avg_interaction_strength(
        contact_matrix_zero,
        f.find_clique_greedy(contact_matrix_zero, s, TTN_BIN, bin_map)
    )
    for s in clique_sizes
]

records = []
for s in clique_sizes:
    for m in models:
        path = f"/Users/ethan/Desktop/chromatin-heart-dev/background_models/greedy_scores_{s}_iterations_1000_{m}.txt"
        scores = [float(x) for x in open(path)]
        records += [{"Clique Size": s, "Score": v, "Model": m.capitalize()} for v in scores]

df = pd.DataFrame(records)

for i, s in enumerate(clique_sizes):
    fig, ax = plt.subplots(figsize=(5, 4))
    sub = df[df["Clique Size"] == s]
    sns.violinplot(data=sub, x="Model", y="Score", inner="quartile", cut=0, ax=ax)
    bodies = [c for c in ax.collections if isinstance(c, PolyCollection)]
    xs = [v for body in bodies for v in body.get_paths()[0].vertices[:, 0]]
    y = greedy_scores[i]
    ax.hlines(y, min(xs), max(xs), color="red", linestyle="--")
    ax.text((min(xs) + max(xs)) / 2, y, f"{y:.6f}", ha="center", va="bottom", color="red", fontweight="bold")
    ax.set_title(f"Clique Size = {s}")
    plt.tight_layout()
    plt.show()

## Stat Testing

In [ ]:
clique_sizes = [3, 4, 5, 8, 10, 12, 15, 20]
p_values = []

for i in clique_sizes:
    with open(f'/Users/ethan/Desktop/chromatin-heart-dev/background_models/greedy_scores_{i}_iterations_1000.txt', 'r') as bg:
        bg_scores = bg.read().splitlines()
        bg_scores = [float(score) for score in bg_scores]

        top_nodes = f.find_clique_greedy(contact_matrix_zero, i, TTN_BIN, bin_map)
        greedy_score = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)

        p_value = t_test(greedy_score, bg_scores)
        p_values.append(p_value)


plt.plot(clique_sizes, p_values, marker='o')
plt.xlabel('Clique Size')
plt.ylabel('p-value')
plt.yscale('log')  # Useful if values vary widely
plt.axhline(0.05, color='r', linestyle='--', label='Significance Threshold (0.05)')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

sizes_strong = [5, 10, 20, 40]
sizes_weak   = [5, 10, 20]

# Compute strong p‑values
p_strong = []
for size in sizes_strong:
    with open(
        f'/Users/ethan/Desktop/chromatin-heart-dev/'
        f'background_models/greedy_scores_{size}_iterations_1000_strong.txt',
        'r'
    ) as strong_bg_file:
        bg_scores_strong = [float(line) for line in strong_bg_file]

    nodes = f.find_clique_greedy(contact_matrix_zero, size, TTN_BIN, bin_map)
    score = f.calculate_avg_interaction_strength(contact_matrix_zero, nodes)
    p_strong.append(empirical_p_value(score, bg_scores_strong))

# Compute weak p‑values
p_weak = []
for size in sizes_weak:
    with open(
        f'/Users/ethan/Desktop/chromatin-heart-dev/'
        f'background_models/greedy_scores_{size}_iterations_1000_weak.txt',
        'r'
    ) as weak_bg_file:
        bg_scores_weak = [float(line) for line in weak_bg_file]

    # reuse the same nodes/score or recompute if you prefer:
    nodes = f.find_clique_greedy(contact_matrix_zero, size, TTN_BIN, bin_map)
    score = f.calculate_avg_interaction_strength(contact_matrix_zero, nodes)
    p_weak.append(empirical_p_value(score, bg_scores_weak))

# Plot both series
plt.figure(figsize=(7,4))
plt.plot(sizes_strong, p_strong, marker='o', linestyle='-', label='Strong')
plt.plot(sizes_weak,   p_weak,   marker='s', linestyle='--', label='Weak')

plt.xlabel('Clique Size')
plt.ylabel('p-value')
plt.yscale('log')
plt.axhline(0.05, color='r', linestyle='--', label='α = 0.05')

# Annotate points
for x, y in zip(sizes_strong, p_strong):
    plt.text(x, y, f"{y:.1e}", ha='center', va='bottom', fontsize=8)
for x, y in zip(sizes_weak, p_weak):
    plt.text(x, y, f"{y:.1e}", ha='center', va='top',    fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def get_stats_from_bg_model(type, clique_size, iterations):
    file_suffix = 'iterations' if type == 'greedy' else 'molecules'
    file_path = f'/Users/ethan/Desktop/chromatin-heart-dev/background_models/{type}_scores_{clique_size}_{file_suffix}_{iterations}.txt'
    
    with open(file_path, 'r') as bg:
        bg_scores = [float(score) for score in bg.read().splitlines()]
    
    top_nodes = f.find_clique_greedy(contact_matrix_zero, clique_size, TTN_BIN, bin_map)
    greedy_score = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)

    mean_bg = np.mean(bg_scores)
    var_bg = np.var(bg_scores)
    std_bg = np.std(bg_scores)

    p_value = mann_whitney_u_test(greedy_score, bg_scores)
    percent_higher = np.mean(np.array(bg_scores) > greedy_score) * 100

    print(f'TTN Clique Score: {greedy_score:.4f}')
    print(f'Background Scores Mean: {mean_bg:.4f}')
    print(f'Background Scores Variance: {var_bg:.4f}')
    print(f'Background Scores Std Dev: {std_bg:.4f}')
    print(f'Percentage of Background Scores Higher Than TTN: {percent_higher:.2f}%')
    print(f'p-value: {p_value:.3e}')

In [ ]:
get_stats_from_bg_model('greedy', 5, 1000)

In [ ]:
get_stats_from_bg_model('greedy', 8, 1000)

In [ ]:
import numpy as np
from scipy.stats import norm

# Given values
ttn_score = 0.0051
mean_bg = 0.0030
std_bg = 0.0019

# Z-score calculation
z_score = (ttn_score - mean_bg) / std_bg

# P-value calculation from Z-score (one-tailed test)
p_value_from_z = 1 - norm.cdf(z_score)

print(f'Z-score: {z_score:.2f}')
print(f'P-value from Z-score: {p_value_from_z:.3e}')

## Alpha selection

To investigate the most optimal alpha for building a clique around TTN, a comparison between strong/weak distributions will be made, looking at how the fold change between the median and mann whitney p value.

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
def random_walk_fast(contact_matrix, start_node, n,
                     neighbors, cdfs,
                     num_molecules=100, alpha=0.1):
    """
    A much faster random‐walk using prebuilt neighbor/CDF lists.
    """
    N = contact_matrix.shape[0]
    visit_count = np.zeros(N, dtype=int)

    for _ in tqdm(range(num_molecules)):
        cur = start_node
        while True:
            visit_count[cur] += 1
            if np.random.rand() < alpha or neighbors[cur].size == 0:
                break
            r = np.random.rand()
            # find next index in CDF
            j = np.searchsorted(cdfs[cur], r, side='right')
            cur = neighbors[cur][j]

    # top-n visited
    return np.argsort(visit_count)[-n:][::-1]

In [ ]:
clique = random_walk_fast(
    contact_matrix_zero, start_node=4275, n=5,
    neighbors=neighbors, cdfs=cdfs,
    num_molecules=5000000, alpha=0.1
)
print(clique)

In [ ]:
import functions as f
from matplotlib.ticker import ScalarFormatter

clique_size     = 10
num_iterations  = 1000
alphas          = [0.01, 0.03, 0.05, 0.1, 0.25, 0.5, 0.75, 0.99]

fold_changes = []
p_values     = []

for alpha in alphas:
    bg_strong = f.create_background_model_rw(
        contact_matrix_zero,
        clique_size,
        bins=gene_bins,
        label='strong',
        num_molecules=1000,
        neighbors=neighbors,
        cdfs=cdfs,
        num_iterations=num_iterations,
        alpha=alpha,
        plot=False
    )
    bg_weak = f.create_background_model_rw(
        contact_matrix_zero,
        clique_size,
        bins=non_gene_bins,
        label='weak',
        num_molecules=1000,
        neighbors=neighbors,
        cdfs=cdfs,
        num_iterations=num_iterations,
        alpha=alpha,
        plot=False
    )

    fold_change = np.median(bg_strong) / np.median(bg_weak)
    _, p_value  = mannwhitneyu(bg_strong, bg_weak, alternative='greater')

    fold_changes.append(fold_change)
    p_values.append(p_value)



fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharex=True)

# fold-Change vs alpha
ax1.plot(alphas, fold_changes, marker='o')
ax1.set_xscale('log')
ax1.set_xticks(alphas)
ax1.get_xaxis().set_major_formatter(ScalarFormatter())
ax1.set_xlabel('Alpha Value')
ax1.set_ylabel('Fold Change')
ax1.set_title('Fold Change vs Alpha')
ax1.grid(True)

# p-value vs alpha with p=0.05 line
ax2.plot(alphas, p_values, marker='s')
ax2.axhline(0.05, linestyle='--', color='red', label='p = 0.05')
ax2.set_xscale('log')
ax2.set_xticks(alphas)
ax2.get_xaxis().set_major_formatter(ScalarFormatter())
ax2.set_xlabel('Alpha Value')
ax2.set_ylabel('Mann–Whitney p-Value')
ax2.set_title('p-Value vs Alpha')


max_p = max(p_values)
top = max(max_p * 1.1, 0.05)
ax2.set_ylim(0, top)

ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Molecules vs Time

In [ ]:
import time
# plot molecules to time
molecules = [100,1000,10000,25000,50000, 100000, 250000]
time_values = []

for m in molecules:
    start = time.time()
    top_nodes = f.random_walk(contact_matrix_zero, TTN_BIN, 5, num_molecules=m, alpha=0.05, verbose=True)
    end = time.time()
    time_values.append(end-start)

plt.plot(molecules, time_values, marker='o')
plt.xlabel('Number of Molecules')
plt.ylabel('Time (s)')
plt.show()


## Alpha vs Time

In [ ]:
# plot alpha values to time using clique size 5
alphas = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
time_values = []

for a in alphas:
    start = time.time()
    top_nodes = f.random_walk(contact_matrix_zero, TTN_BIN, 5, num_molecules=1000, alpha=a, verbose=True)
    end = time.time()
    time_values.append(end-start)

plt.plot(alphas, time_values, marker='o')
plt.xlabel('Alpha Value')
plt.ylabel('Time (s)')
plt.show()



## Alpha vs p value, (for now can use clique size 5)

In [ ]:
top_nodes

In [ ]:
top_nodes

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt

alphas = [0.1, 0.2, 0.3, 0.5, 0.99]
p_values = []

for a in alphas:
    top_nodes = f.random_walk(contact_matrix_zero, TTN_BIN, 5, num_molecules=1000, alpha=a, verbose=True)
    rw_score = f.calculate_avg_interaction_strength(contact_matrix_zero, top_nodes)
    bg_model_rw = f.create_background_model_rw_strong(contact_matrix_zero, 5, gene_bins=gene_bins, num_molecules=1000, num_iterations=1000, alpha=a)

    p_value = mann_whitney_u_test(rw_score, bg_model_rw)
    p_values.append(p_value)

plt.plot(alphas, p_values, marker='o')
plt.xlabel('Reset Probability (alpha)')
plt.ylabel('p-value')
plt.yscale('log')  # Log scale if values vary widely
plt.axhline(0.05, color='r', linestyle='--', label='Significance Threshold (0.05)')
plt.legend()
plt.show()

## Bed Narrow Peak

In [ ]:
import functions as f
# LOAD BIN MAP
bin_map_loc = f.load_bin_map_loc('/Users/ethan/Desktop/chromatin-heart-dev/data/bin_map_human_100000.bed')

In [ ]:
f.find_bin('chr2', 61443502, bin_map_loc)

In [ ]:
# Open data/transcription_factors, contains a directory of .bed files containing transcription factor binding sites

import os
import pandas as pd
import seaborn as sns

# Load the transcription factor binding sites
data_dir = './data/transcription_factors'
tf_files = os.listdir(data_dir)

# Return counts of transcription factor binding sites in each 100kb bin
def process_bed_file(file_path):
    bin_counts = {}

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('#') or not line.strip():
                continue
            columns = line.split()
            
            chrom = columns[0]
            start_position = int(columns[1])    
            
            bin_number = f.find_bin(chrom, start_position, bin_map_loc)
            
            if bin_number not in bin_counts:
                bin_counts[bin_number] = 0
            bin_counts[bin_number] += 1

    return bin_counts

interaction_scores = []

# Contains top 1 bin for each TF
max_bins = []
bin_set = set()

for filename in os.listdir(data_dir):
    # open file

    try:    
        bin_counts = process_bed_file(os.path.join(data_dir, filename))
    except:
        print(f'Error processing {filename}')
        continue

    # get top 20 bins
    top_bins = sorted(bin_counts, key=bin_counts.get, reverse=True)[:20]

    max_bins.append(top_bins[0])
    bin_set.update(top_bins)

    interaction_strength = f.calculate_avg_interaction_strength(contact_matrix_zero, top_bins)
    interaction_scores.append(interaction_strength)

    # get span of chromosmes of the top bins
    chrom_span = f.get_chromosome_span(top_bins, bin_map)



# Plot the distribution as a violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(y=interaction_scores, inner='box', color='skyblue')
plt.ylabel('Average Interaction Score')
plt.title('Distribution of Interaction Strength for TF-Clique')
plt.show()
    


In [ ]:
max(interaction_scores)

In [ ]:
max_bins

In [ ]:
interaction_scores = []

for i in tqdm(range(len(max_bins))):
    clique = f.find_clique_greedy(contact_matrix_zero, 5, max_bins[i], bin_map)
    score = f.calculate_avg_interaction_strength(contact_matrix_zero, clique)
    interaction_scores.append(score)

# Plot the distribution of interaction scores
plt.figure(figsize=(10, 6))
plt.hist(interaction_scores, bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Average Interaction Score')
plt.ylabel('Frequency')
plt.title(f'Distribution of Interaction Strength for TF-Clique')
plt.show()



    

In [ ]:
interaction_scores = []

for i in tqdm(range(len(max_bins))):
    clique = f.random_walk(contact_matrix_zero, max_bins[i], 5, num_molecules=1000, alpha=0.05, verbose=False)
    score = f.calculate_avg_interaction_strength(contact_matrix_zero, clique)
    interaction_scores.append(score)

# Plot the distribution of interaction scores
plt.figure(figsize=(10, 6))
plt.hist(interaction_scores, bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Average Interaction Score')
plt.ylabel('Frequency')
plt.title(f'Distribution of Interaction Strength for TF-Clique')
plt.show()



    

In [ ]:
ttn_clique = f.random_walk(contact_matrix_zero, TTN_BIN, 5, num_molecules=1000, alpha=0.05, verbose=False)
score_rw = f.calculate_avg_interaction_strength(contact_matrix_zero, ttn_clique)

t_test(score_rw, interaction_scores)

## Genes from TTN Clique

In [ ]:
import functions as f

In [ ]:
# 4275, 4276, 4277, 4278

In [ ]:
top_40_clique = f.find_clique_greedy(contact_matrix_zero, 40, TTN_BIN, bin_map)

In [ ]:
fdasdsas = f.find_clique_greedy(contact_matrix_zero, 40, TTN_BIN, bin_map)

In [ ]:
top_40_clique

In [ ]:
from tqdm import tqdm

TTN_GENES = set()
for bin in tqdm(top_40_clique):
    genes = f.find_gene_from_bin(bin, '/Users/ethan/Desktop/chromatin-heart-dev/data/bin_map_human_100000.bed', '/Users/ethan/Desktop/chromatin-heart-dev/data/gencode.v38.annotation.gtf')
    TTN_GENES.update(genes)

In [ ]:
top_40_clique_rw = f.random_walk(contact_matrix_zero, TTN_BIN, 40, num_molecules=1000, alpha=0.05, verbose=True)

In [ ]:
top_40_clique_rw

In [ ]:
TTN_GENES_RW = set()    

for bin in tqdm(top_40_clique_rw):
    genes = f.find_gene_from_bin(bin, '/Users/ethan/Desktop/chromatin-heart-dev/data/bin_map_human_100000.bed', '/Users/ethan/Desktop/chromatin-heart-dev/data/gencode.v38.annotation.gtf')
    TTN_GENES_RW.update(genes)

In [ ]:
len(TTN_GENES)

In [ ]:
# write list to file

with open('/Users/ethan/Desktop/chromatin-heart-dev/TTN_genes.txt', 'w') as tg:
    for item in TTN_GENES:
        tg.write("%s\n" % item)




In [ ]:
import functions as f

from time import time

start = time()


f.random_walk_fast(
    contact_matrix_zero, TTN_BIN, 5,
    neighbors=neighbors, cdfs=cdfs,
    num_molecules=10000, alpha=0.05
)
end = time()

print(f"Time taken: {end - start} seconds")